# Лабораторная работа №5

### Выполнили: 
* Иванов Александр Константинович, 368220 
* Братушка Никита Игоревич, 367924
* Проворов Николай Дмитриевич, 368690


# Импортирование необходимых библиотек

In [28]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import pairplot, heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, f1_score, classification_report

## Загрузим датасет для классификации и разделим его в соотношении 80/20 на обучающую и тестовую выборки соответственно:

In [16]:
train = pd.read_csv(open("heart.csv"))

print(train.head()) # Выводим считанные данные

X = train.drop("target", axis=1)
y = train["target"]

X_train, X_tests, y_train, y_tests = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Обучающая выборка
X_tests = scaler.fit_transform(X_tests) # Тестовая выборка

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  ca  thal  target
0   63    1   3       145   233    1        0      150      0      2.3      0   0     1       1
1   37    1   2       130   250    0        1      187      0      3.5      0   0     2       1
2   41    0   1       130   204    0        0      172      0      1.4      2   0     2       1
3   56    1   1       120   236    0        1      178      0      0.8      2   0     2       1
4   57    0   0       120   354    0        1      163      1      0.6      2   0     2       1


## Бэггинг решающих деревьев(случайный лес, RandomForest)

С помощью удобного инструмента GridSearchCV выберем лучшие параметры для модели

In [31]:
classifier_rf = RandomForestClassifier(random_state=10)
param_rf = {
    "n_estimators": [20, 50, 100, 200],
    "max_depth" : [None, 5, 10, 20],
    "min_samples_split" : [2, 5, 10, 20]
}

grid_search_rf = GridSearchCV(estimator=classifier_rf, param_grid=param_rf, scoring="accuracy", cv=5)

# Обучение
grid_search_rf.fit(X_train, y_train)

# Предсказания модели на основе лучших параметров из grid_search_rf.best_estimator_ (уже обучена)
y_predict_rf = grid_search_rf.best_estimator_.predict(X_tests) 


In [32]:
# Параметры
print("Лучшие параметры: ")
print("Количество деревьев(n_estimators): ", grid_search_rf.best_params_.get("n_estimators"))
print("Максимальная глубина деревьев (max_depth): ", grid_search_rf.best_params_.get("max_depth"))
print("Количество образцов в узле (min_samples_split): ", grid_search_rf.best_params_.get("min_samples_split"), "\n")

# Оценка
print("Отчет о классификации:\n", classification_report(y_true=y_tests, y_pred=y_predict_rf))
print("Точность(accuracy): ", accuracy_score(y_true=y_tests, y_pred=y_predict_rf))
print("F1 (f1_score): ", f1_score(y_true=y_tests, y_pred=y_predict_rf))

Лучшие параметры: 
Количество деревьев(n_estimators):  200
Максимальная глубина деревьев (max_depth):  5
Количество образцов в узле (min_samples_split):  10 

Отчет о классификации:
               precision    recall  f1-score   support

           0       0.96      0.72      0.82        32
           1       0.76      0.97      0.85        29

    accuracy                           0.84        61
   macro avg       0.86      0.84      0.83        61
weighted avg       0.86      0.84      0.83        61

Точность(accuracy):  0.8360655737704918
F1 (f1_score):  0.8484848484848485


## Градиентный бустинг над решающими деревьями (GradientBoostingRegressor)

In [35]:
classifier_gb = GradientBoostingClassifier(random_state=10)
param_gb = {
    "n_estimators": [20, 50, 100, 200],
    "max_depth" : [None, 5, 10, 20],
    "min_samples_split" : [2, 5, 10, 20]
}

grid_search_gb = GridSearchCV(estimator=classifier_gb, param_grid=param_gb, scoring="accuracy", cv=5)

# Обучение
grid_search_gb.fit(X_train, y_train)

# Предсказания модели на основе лучших параметров из grid_search_gb.best_estimator_ (уже обучена)
y_predict_gb = grid_search_gb.best_estimator_.predict(X_tests) 


In [36]:
# Параметры
print("Лучшие параметры: ")
print("Количество деревьев(n_estimators): ", grid_search_gb.best_params_.get("n_estimators"))
print("Максимальная глубина деревьев (max_depth): ", grid_search_gb.best_params_.get("max_depth"))
print("Количество образцов в узле (min_samples_split): ", grid_search_gb.best_params_.get("min_samples_split"), "\n")

# Оценка
print("Отчет о классификации:\n", classification_report(y_true=y_tests, y_pred=y_predict_gb))
print("Точность(accuracy): ", accuracy_score(y_true=y_tests, y_pred=y_predict_gb))
print("F1 (f1_score): ", f1_score(y_true=y_tests, y_pred=y_predict_gb))

Лучшие параметры: 
Количество деревьев(n_estimators):  50
Максимальная глубина деревьев (max_depth):  5
Количество образцов в узле (min_samples_split):  20 

Отчет о классификации:
               precision    recall  f1-score   support

           0       0.95      0.59      0.73        32
           1       0.68      0.97      0.80        29

    accuracy                           0.77        61
   macro avg       0.82      0.78      0.77        61
weighted avg       0.82      0.77      0.76        61

Точность(accuracy):  0.7704918032786885
F1 (f1_score):  0.8
